In [1]:
from scripts.experiments.symmetries import symplectic_pauli_reduction, pauli_reduce, symmetric_symplectic
from quaos.models.Ising import ising_chain_hamiltonian, ising_2d_hamiltonian

In [8]:



    N = 3
    J = 1
    h = 1
    H = ising_chain_hamiltonian(N, J, h, periodic=True)

    h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(H)
    print(h_red)
    print(conditioned_hamiltonians[0])


(1+0j)|x0z1 x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 x0z0 | 0 
(1+0j)|x0z1 x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 x0z0 | 0 
(1+0j)|x1z0 x1z0 x0z0 | 0 
(1+0j)|x0z0 x1z0 x0z1 | 0 

(1+0j)|x1z0 x1z0 | 0 
(1+0j)|x1z0 x0z0 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x0z1 | 0 
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 



In [10]:
F = symmetric_symplectic(H, max_cycle=200)

1
Checking permutation [(0, 1)]
2
Checking permutation [(3, 4)]
3
Checking permutation [(3, 5)]
4
Checking permutation [(4, 5)]
5
Checking permutation [(3, 4), (3, 5)]
6
Checking permutation [(3, 5), (3, 4)]
7
Checking permutation [(0, 1), (3, 4)]
8
Checking permutation [(0, 1), (3, 5)]
9
Checking permutation [(0, 1), (4, 5)]
10
Checking permutation [(0, 1), (3, 4), (3, 5)]
11
Checking permutation [(0, 1), (3, 5), (3, 4)]
12


Exception: No valid permutation found

In [15]:
n_x = 3
n_y = 3
J_zz = 1
h_x = 1
H_2d = ising_2d_hamiltonian(n_x, n_y, J_zz, h_x, periodic=False)

h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(H_2d)
print(conditioned_hamiltonians[0].n_paulis())

21


In [16]:
F = symmetric_symplectic(conditioned_hamiltonians[0], max_cycle=200)

1
Checking permutation [(0, 1)]
2
Checking permutation [(0, 2)]
3
Checking permutation [(0, 3)]
4
Checking permutation [(0, 4)]
5
Checking permutation [(0, 5)]
6
Checking permutation [(0, 6)]
7
Checking permutation [(0, 7)]
8
Checking permutation [(1, 2)]
9
Checking permutation [(1, 3)]
10
Checking permutation [(1, 4)]
11
Checking permutation [(1, 5)]
12
Checking permutation [(1, 6)]
13
Checking permutation [(1, 7)]
14
Checking permutation [(2, 3)]
15
Checking permutation [(2, 4)]
16
Checking permutation [(2, 5)]
17
Checking permutation [(2, 6)]
18
Checking permutation [(2, 7)]
19
Checking permutation [(3, 4)]
20
Checking permutation [(3, 5)]
21
Checking permutation [(3, 6)]
22
Checking permutation [(3, 7)]
23
Checking permutation [(4, 5)]
24
Checking permutation [(4, 6)]
25
Checking permutation [(4, 7)]
26
Checking permutation [(5, 6)]
27
Checking permutation [(5, 7)]
28
Checking permutation [(6, 7)]
29
Checking permutation [(9, 10)]
30
Checking permutation [(11, 13)]
31
Checking perm

KeyboardInterrupt: 